Tool Auto Export Data from Purchase Info Record Template to XML & upload to LTMC

In [11]:
# Getting and Knowing Data
import pandas as pd
import numpy as np
from datetime import datetime
import time

PURCHASE INFO RECORD DATA

In [12]:
# Step 1: Import Dataset
file_excel = r"C:\Users\Admin\Desktop\T12.L16.SAP_MM_Form_Purchase Info Record EA AF.xlsx"
df_data = pd.read_excel(file_excel, usecols="A:L", skiprows=[1], keep_default_na=False)
df_data['KUMNE'] = df_data['KUMNE'].astype(int)
df_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   MATNR      106 non-null    object
 1   LIFNR      106 non-null    int64 
 2   EKORG      106 non-null    int64 
 3   WERKS      106 non-null    int64 
 4   MEINS      106 non-null    object
 5   KBETR_EXT  106 non-null    int64 
 6   KPEIN      106 non-null    int64 
 7   KMEIN      106 non-null    object
 8   KUMNE      106 non-null    int32 
 9   KUMZA      106 non-null    int64 
 10  DATAB      106 non-null    object
 11  DATBI      106 non-null    object
dtypes: int32(1), int64(6), object(5)
memory usage: 9.6+ KB


LTMC_PIR_XMLSHEET.


In [13]:
df_general = pd.DataFrame(columns=["INFNR","MATNR","MATKL","LIFNR","TXZ01","SORTL","MEINS","UMREZ","UMREN","IDNLF","VERKF","TELF1","MAHN1","MAHN2","MAHN3","LMEIN","VABME","WGLIF","KOLIF","URZLA","REGIO","LIFAB","LIFBI","RELIF"])
df_purch_org = pd.DataFrame(columns=["INFNR","EKORG","ESOKZ","WERKS","EKGRP","WAERS","MINBM","NORBM","APLFZ","UEBTO","UEBTK","UNTTO","BWTAR","MTXNO","WEBRE","KZABS","MWSKZ","EVERS","BSTAE","XERSN","MHDRZ","IPRKZ","BSTMA","EXPRF_LAND1","EXPRF","RDPRF","VENDOR_RMA_REQ","AUT_SOURCE","SKTOF","MEPRF","INCOV","INCO1","INCO2_L","INCO3_L"])
df_conditions = pd.DataFrame(columns=["INFNR","EKORG","ESOKZ","WERKS","KOPOS","COND_TYPE","VALID_FROM","VALID_TO","KRECH","KBETR_EXT","KONWA","KPEIN","KMEIN","KUMZA","KUMNE","MEINS"])

dfs = [df_general,df_purch_org,df_conditions]
sheets = ['General Data','Purchasing Organization Data','Conditions'] 
# Convert Data from Template.
for field in df_data.columns:
    for df in dfs:
        if field in list(df.columns):
            df[field] = df_data[field]
# Create sequence number
seq = str(datetime.now().strftime('%Y%m%d%H%M')) + "-"
for index, row in df_data.iterrows():
    for df in dfs:
        # Legacy Number
        df.at[index, 'INFNR'] = str(seq + str(index+1))

Defauld Data in template.

In [14]:
# Purchasing Org Data.
df_purch_org['ESOKZ'] = 3   # Info Record Category.
df_purch_org['EKGRP'] = 105
df_purch_org['WAERS'] = "VND"   # Currency.
df_purch_org['NORBM'] = 1    # Standard Purchase Order Qty.
df_purch_org['BWTAR'] = "EX"    # Val Type.
df_purch_org['WEBRE'] = "X"    # "Indicator: GR-Based Invoice Verification
df_purch_org['MWSKZ'] = "VA"    # Tax on sales/purchases code
df_purch_org['BSTAE'] = "0004"
df_purch_org['APLFZ'] = '1'
# Conditions Data.

df_conditions['COND_TYPE'] = "PB00"
df_conditions['ESOKZ'] = 3   # Info Record Category.
df_conditions['KOPOS'] = '01'   # Condition Sequence Number
df_conditions['KONWA'] = "VND"
df_conditions['VALID_FROM'] = df_data['DATAB']
df_conditions['VALID_TO']= df_data['DATBI']


EXPORT TO EXCEL FILE

Put multiple dataframes across separate tabs/sheets

<!-- # Defauld Data in template. -->

In [15]:
# function
def dfs_tabs(df_list, sheet_list, file_name):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0, index= False)   
    writer.close()

# run function
dfs_tabs(dfs, sheets, "LTMC.xlsx")